# PA Opioid Epidemic

In [567]:
import pandas as pd
import numpy as np
import matplotlib as plt
import folium

In [568]:
overdose_dat = 'Overdose_Information_Network_Data_CY_January_2018_-_Current_Monthly_County_State_Police.csv'
county_geo = 'PaCounty2020_01.geojson'

In [569]:
ovd = pd.read_csv(overdose_dat)

In [570]:
df = ovd.drop(['Response Time Desc', 'Revive Action Desc', 'Response Desc', 'Victim County', 'Victim OD Drug ID', 'Third Party Admin Desc', 'Administration ID', 'Dose Count', 'Dose Unit', 'Dose Desc', 'Incident ID', 'Victim State', 'Incident State', 'Victim County Latitude and Longitude', 'Incident County Latitude and Longitude', 'Victim ID', 'Victim County FIPS Code', 'Incident County FIPS Code', 'Victim County Latitude', 'Victim County Longitude'], axis=1)
df = df.rename(columns={"Survive": "Survived", "Ethnicity Desc": "Hispanic", "Accidental Exposure": "Accident", "Age Range": "Age", "Naloxone Administered": "Naloxone", "Incident County Longitude": "Longitude", "Incident County Latitude": "Latitude", "Susp OD Drug Desc": "Drug", "Gender Desc": "Gender", "Incident Date": "Date", "Incident Time": "Time", "Incident County Name": "County"})
df['Date'] = df['Date'] + ":" + df['Time']
df = df.drop(['Time'], axis=1)

In [571]:
df['Date'] = df['Date'].replace(to_replace=r'\/', value=':', regex=True)
df['Date'] = pd.to_datetime(df['Date'], format='%m:%d:%Y:%H:%M:%S')

In [572]:
df['Age'] = df['Age'].replace(to_replace=r'\s\-\s', value='-', regex=True)
df['Age'] = df['Age'].replace(to_replace=r'14-Oct', value='0-14')
df['Age'] = df['Age'].replace(to_replace=r'0-9', value='0-14')
df['Age'] = df['Age'].replace(to_replace=r'70-79', value='70-99')
df['Age'] = df['Age'].replace(to_replace=r'80-*', value='70-99')

In [573]:
df['Gender'] = df['Gender'].replace(to_replace=r'Male', value='1')
df['Gender'] = df['Gender'].replace(to_replace=r'Female', value='2')
df['Gender'] = df['Gender'].replace(to_replace=r'Unknown', value='0')

In [574]:
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Not Hispanic', value='0')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Hispanic', value='1')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Unknown', value='0')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Mongolian', value='0')

In [575]:
df['Race'] = df['Race'].replace(to_replace=r'Unknown', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'Asian or Pacific Islander', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'American Indian or Alaskan Native', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'Other', value='0')
df['Race'] = df['Race'].replace(to_replace=r'White', value='1')
df['Race'] = df['Race'].replace(to_replace=r'Black', value='2')

In [576]:
df['Drug'] = df['Drug'].str.lower()
df['Drug'] = df['Drug'].replace(to_replace=r'benzodiazepines (i.e.valium, xanax, ativan, etc)', value='benzodiazepines')
df['Drug'] = df['Drug'].replace(to_replace=r'buprenorphine (i.e. suboxone, sublocade, subutex, butrans, buprenex, etc)', value='buprenorphine')
df['Drug'] = df['Drug'].replace(to_replace=r'barbiturates (i.e. amytal, nembutal, etc)', value='barbiturates')
df['Drug'] = df['Drug'].replace(to_replace=r'fentanyl analog/other synthetic opioid', value='fentanyl')

In [577]:
df['Accident'] = df['Accident'].replace(to_replace=r'N', value='0')
df['Accident'] = df['Accident'].replace(to_replace=r'Y', value='1')

In [578]:
df['Naloxone'] = df['Naloxone'].replace(to_replace=r'N', value='0')
df['Naloxone'] = df['Naloxone'].replace(to_replace=r'Y', value='1')

In [579]:
df['Survived'] = df['Survived'].replace(to_replace=r'N', value='0')
df['Survived'] = df['Survived'].replace(to_replace=r'Y', value='1')
df['Survived'] = df['Survived'].replace(to_replace=r'U', value='2')

In [580]:
df['Day'] = df['Day'].replace(to_replace=r'Sunday', value='0')
df['Day'] = df['Day'].replace(to_replace=r'Monday', value='1')
df['Day'] = df['Day'].replace(to_replace=r'Tuesday', value='2')
df['Day'] = df['Day'].replace(to_replace=r'Wednesday', value='3')
df['Day'] = df['Day'].replace(to_replace=r'Thursday', value='4')
df['Day'] = df['Day'].replace(to_replace=r'Friday', value='5')
df['Day'] = df['Day'].replace(to_replace=r'Saturday', value='6')

In [581]:
df['Age'].value_counts()

30-39    8755
25-29    4865
40-49    3937
20-24    2678
50-59    2165
60-69     683
15-19     470
70-99      87
0-14       68
Name: Age, dtype: int64

In [583]:
df['Drug'].value_counts()

heroin                      12201
fentanyl                     4085
unknown                      2433
pharmaceutical opioid         784
cocaine/crack                 624
other                         545
methamphetamine               521
alcohol                       482
pharmaceutical other          469
benzodiazepines               464
marijuana                     339
synthetic marijuana           257
buprenorphine                 184
carfentanil                   125
methadone                     110
pharmaceutical stimulant       36
barbiturates                   34
bath salts                     15
Name: Drug, dtype: int64

In [582]:
df.head()

,Date,Day,County,Gender,Age,Race,Hispanic,Accident,Drug,Naloxone,Survived,Latitude,Longitude
0,2020-07-02 12:48:00,4,York,1,50-59,1,1,0,heroin,1,1,39.921925,-76.725761
1,2020-09-20 23:11:00,0,York,1,20-24,1,0,0,fentanyl,1,1,39.921925,-76.725761
2,2020-09-11 20:00:00,5,Union,1,50-59,1,0,0,heroin,0,0,40.964731,-77.064158
3,2020-07-10 21:10:00,5,York,1,40-49,1,0,0,fentanyl,1,1,39.921925,-76.725761
4,2020-06-20 20:52:00,6,York,1,40-49,1,0,0,fentanyl,0,0,39.921925,-76.725761
